<a href="https://colab.research.google.com/github/minhyeok2487/Deep_Learning/blob/main/Keras_Mnist_Dataset_With_Simple_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Keras Mnist Dataset With Simple RNN

**Test accuracy: 47.2%**

## Step 1. Library






In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense
from keras.models import Sequential

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
from IPython.display import Image

warnings.filterwarnings('ignore')
%matplotlib inline

SEED = 34

## Step 2. Load Data

In [ ]:
from keras.datasets import mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()

In [ ]:
# Check train data shape
print(x_train.shape)
print(y_train.shape)

# Count the number of unique train labels
unique, counts = np.unique(y_train, return_counts=True)
print("Train labels: ", dict(zip(unique, counts)))

In [ ]:
# Check test data shape
print(x_test.shape)
print(y_test.shape)

# Count the number of unique test labels
unique, counts = np.unique(y_test, return_counts=True)
print("Test labels: ", dict(zip(unique, counts)))

In [ ]:
# Show data example
plt.figure()
plt.imshow(x_train[3])
plt.colorbar()
plt.grid(True)
plt.show()
print("\nThe number is", y_train[3])

In [ ]:
# Outputs all nonzero values for one image 
list(filter(lambda x: x != 0, x_train[3].reshape(-1)))[:10]

In [ ]:
# Check dtype
print(x_train.dtype)
print(y_train.dtype)

## Step 3. Data Preprocessing

### 3.1 train/test data check range

In [ ]:
print(list(filter(lambda x: x != 0, x_train[3].reshape(-1)))[:10])
print(list(filter(lambda x: x != 0, y_train.reshape(-1)))[:10])

print(list(filter(lambda x: x != 0, x_test[3].reshape(-1)))[:10])
print(list(filter(lambda x: x != 0, y_test.reshape(-1)))[:10])

### 3.2 Min/Max values of train/test data 

In [ ]:
print(max(x_train.reshape(-1)), min(x_train.reshape(-1)))
print(max(x_test.reshape(-1)), min(x_test.reshape(-1)))

### 3.3 Data Normalize

In [ ]:
# Convert uint8 to float64
x_train = x_train.astype(np.float64)
x_test = x_test.astype(np.float64)

# 0~1 Normalize
x_train = x_train / 255
x_test = x_test / 255

print(list(filter(lambda x: x != 0, x_train[3].reshape(-1)))[:10])
print(list(filter(lambda x: x != 0, y_train.reshape(-1)))[:10])

print(list(filter(lambda x: x != 0, x_test[3].reshape(-1)))[:10])
print(list(filter(lambda x: x != 0, y_test.reshape(-1)))[:10])

## Step 4. Data Visualization

In [ ]:
indexes = np.random.randint(0, x_train.shape[0], size=25)
images = x_train[indexes]
labels = y_train[indexes]


# plot the 25 mnist digits
plt.figure(figsize=(5,5))
for i in range(len(indexes)):
    plt.subplot(5, 5, i + 1)
    image = images[i]
    plt.imshow(image, cmap='gray')
    plt.axis('off')
   
plt.show()
plt.savefig("mnist-samples.png")
plt.close('all')

## Step 5. Data augmentation - Noise

In [ ]:
noisy_image = x_train[3] + np.random.normal(loc = 0.5, scale=0.1, size=(28,28))
plt.imshow(noisy_image, cmap='gray')
plt.colorbar()
plt.show()

In [ ]:
x_train_noisy = x_train + np.random.normal(loc = 0.5, scale=0.1, size=x_train.shape)
x_test_noisy = x_test + np.random.normal(loc = 0.5, scale=0.1, size=x_test.shape)

x_train_noisy[x_train_noisy > 1.0] = 1.0 
x_test_noisy[x_test_noisy > 1.0] = 1.0

In [ ]:
plt.imshow(x_train_noisy[:5].transpose( (1, 0, 2) ).reshape(28, -1), cmap='gray')
plt.colorbar()
plt.show()
print(y_train[:5])

## Step 6. Designing Model

### 6.1 One-Hot Encoding

In [ ]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
print(y_train.shape, y_test.shape)

### 6.2 RNN Model

In [ ]:
from keras.layers import SimpleRNN
from keras.layers import Dense, Input
from keras.models import Model

inputs = Input(shape=(28,28))
x1 = SimpleRNN(64, activation="tanh")(inputs)
x2 = Dense(10, activation="softmax")(x1)
model = Model(inputs, x2)

In [ ]:
model.summary()

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics= ["accuracy"])

In [ ]:
hist = model.fit(x_train_noisy, y_train,
          validation_data=(x_test_noisy, y_test), epochs=20, verbose=2)

In [ ]:
plt.plot(hist.history['accuracy'], label='accuracy')
plt.plot(hist.history['loss'], label='loss')
plt.plot(hist.history['val_accuracy'], label='val_accuracy')
plt.plot(hist.history['val_loss'], label='val_loss')
plt.legend(loc="upper left")
plt.show

## Step 7. Save Model

In [ ]:
model.save("./MNIST_RNN_01.h5")

In [ ]:
new_model = tf.keras.models.load_model("./MNIST_RNN_01.h5")

In [ ]:
res = new_model.predict(x_test_noisy[4:5])
res.shape
plt.bar(range(10), res[0], color='red')
plt.bar(np.array(range(10))+ 0.35, y_test[3])
plt.show()

In [ ]:
loss, acc = model.evaluate(x_test, y_test, verbose=2)
print("\nTest accuracy: %.1f%%" % (100.0 * acc))

In [ ]:
from google.colab import files

files.download("./MNIST_RNN_01.h5")